# Empirical Project 3

## Python-specific learning objectives

In addition to the learning objectives for this project, in this section you will learn how to TODO

## Getting started in Python

TODO (list packages needed here)

## Python Walkthrough 3.1

**Importing the datafile into Python**

First, we need to import the packages and settings we'll be using:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from plotnine import *
import seaborn as sns
import pingouin as pg
import numpy as np
from pathlib import Path
import warnings

# Set the plot style for prettier charts:
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)
plt.rcParams["figure.figsize"] = [6, 3]
plt.rcParams["figure.dpi"] = 150

# Ignore warnings to make nice output
warnings.simplefilter("ignore")

The data are stored in `.xlsx` format, so we use the `pd.read_excel` function from the **pandas** package to import it. We also import the **matplotlib** library as this includes packages that we will use later for drawing charts.

If you open the data in Microsoft's Excel, Apple's Numbers, or LibreOffice's Calc (free), you will see that there are two tabs: ‘Data Dictionary’, which contains some information about the variables, and ‘Data’, which contains the actual data. Let’s import both into Python, so that we do not need to refer back to Excel for the additional information about variables.

Remember to store the downloaded file in a subdirectory of your current working directory called `data`. Let's load up the data.

In [ ]:
path_to_data = Path("data/Dataset Project 3.xlsx")

var_info = pd.read_excel(path_to_data, sheet_name="Data Dictionary")
data = pd.read_excel(path_to_data, sheet_name="Data")

Let's use the `info()` method to ensure that all the variables were given the correct *datatypes*:

In [ ]:
data.head()

In [ ]:
data.info()

Python classified all the variables containing numbers as either `int64` (integers) or `float64` (real numbers). However, other variables got labelled as `object` because they don't numbers, and some of the columns labelled as numbers are actually categorical variables.

Let's ensure that `type`, `taxed`, `supp`, `store_id`, `store_type`, `type2` and `product_id` are represented as categorical variables. We'll use the `astype` function to convert these columns.

In [ ]:
for col in ["type", "taxed", "supp", "store_id", "store_type", "type2", "product_id"]:
    data[col] = data[col].astype("category")

# rename some of the categories
data["taxed"] = data["taxed"].cat.rename_categories(["not taxed", "taxed"])
data["supp"] = data["supp"].cat.rename_categories(["standard", "supplemental"])
data["store_type"] = data["store_type"].cat.rename_categories(["Large supermarket", "Small supermarket", "Pharmacy", "Gas station"])

You can see that we used `.cat.rename_categories` to give more meaningful names to different categories (where they are clearly defined).

There is another variable, `time`, which is classified as having object datatype, but should be a categorical variable. Before we do this, we use the unique command to check the categories of this variable.

In [ ]:
data["time"].unique()

If you look at the timeline in the [Silver et al. (2017) paper](https://doi.org/10.1371/journal.pmed.1002283), you will notice that the third price survey was in March 2016, not in March 2015, so the data has been labelled incorrectly. We shall therefore change all the values `MAR2015` to `MAR2016`.

In [ ]:
data.loc[data["time"]=="MAR2015", "time"] = "MAR2016"

We can now change `time` into a categorical variable.

In [ ]:
data["time"] = data["time"].astype("category")

## Python Walkthrough 3.2

**Counting the number of unique elements in a variable**

We use two functions here: unique to obtain a list of the unique elements of the variables of interest (`data["store_id"]` and `data["product_id"]`), then `len` to count how long the list is. We will create two variables, `no_stores` and `no_products`, that contain the number of stores and products respectively.

In [ ]:
no_stores = len(data["store_id"].unique())
no_products = len(data["product_id"].unique())
print(f"The {no_stores=} while the {no_products=}.")

## Python Walkthrough 3.3

**Creating frequency tables**

### Frequency table for store type and time period.

We use the `pd.crosstab` function to perform a cross-tabulation count of `store_type` by `time`.

In [ ]:
pd.crosstab(index=data["store_type"], columns=data["time"])

There are fewer observations taken from gas stations and pharmacies and more from small supermarkets.

### Frequency table for store type and taxed

Now we repeat the steps above to make a frequency table with `store_type` as the row variable and `taxed` as the column variable. We pass a list of dataframe columns to the `pd.crosstab` function because we also want separate values for each time period. To get the total row each row and column, we add `margins=True`.

In [ ]:
pd.crosstab(index=data["store_type"], columns=[data["time"], data["taxed"]], margins=True)

### Frequency table for product type and time period

Now we make a frequency table with product type (`type`) as the row variable and time period (`time`) as the column variable.

In [ ]:
pd.crosstab(index=data["type"], columns=data["time"], margins=True)

This table shows that there were no observations for the category `Sport-diet` in March 2016. As this is a drink which even in the other months has very few observations, it may be a product that is offered only in one shop, and it is possible that this shop was not visited in March 2016. The product may also be a seasonal product that is not available in March. It is also likely that Water-sweet is offered in only one shop.

## Python Walkthrough 3.4

**Calculating conditional means**

Calculating conditional, or group, means is a common data cleaning operation you will encounter. It can be a bit confusing, but, in the typical use, case we might just be asking what the mean is for each period (instead of the mean for all periods). Furthermore, we must choose whether to bring the mean back into our data with the *original index* (and so the original number of rows) or with a new index that reflects what the mean is conditional on. We'll now see one way to do this.

In order to identify products (`"product_id"`) that have observations for all three periods (`"DEC2014"`, `"JUN2015"` and `"MAR2016"`), we will first create a new variable called `period_test`, which takes the value `True` if we have observations in all periods for a product in a particular store, and `False` otherwise. These true/false variables are called ‘boolean variables’.

This is going to be a complex operation, so hold on to your hats. First we need to group everything by `"product_id"` and `"store_id"` using a `groupby` operation. Then we'll select the column we want, `"time"`. Then we'll use `transform`: this returns a series with the same number of rows as the original index, so will fit nicely into our existing data frame (as opposed to having the index be set to the number of unique groups from the groupby). Within our transform, we'll ask for the number of unique (`nunique`) categories of time. For entries that exist, this will be 1, 2, or 3 depending on whether the product is available in 1, 2, or 3 of the periods. For entries that don't exist, this comes back as `NaN`, which means not a number. So we then use `.fillna(0)` to set these to zero. Finally we compare this object against the known unique number of different periods (3). 

In [ ]:
data.groupby(["product_id", "store_id"])["time"].transform(lambda x: x.nunique()).fillna(0) == data["time"].nunique()

*Tip* The [**pandas** documentation](https://pandas.pydata.org/docs/index.html) provides lots of detailed examples of all of its extensive functionality, including the [`transform`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.transform.html?highlight=transform#pandas.core.groupby.DataFrameGroupBy.transform) function.

Let's store this in the dataframe as a new column:

In [ ]:
data["period_test"] = data.groupby(["product_id", "store_id"])["time"].transform(lambda x: x.nunique()).fillna(0) == data["time"].nunique()

Now we can check this worked by picking out the first unique product_id, store_id combination:

In [ ]:
data.loc[(data["product_id"] == 29) & (data["store_id"] == 16), :]

`period_test` reports `True` for this combination, and we cn see that all three periods are present. Let's try another one:

In [ ]:
data.loc[(data["product_id"] == 29) & (data["store_id"] == 10), :]

In this case, the product-store cell is only there for two out of the three periods and so gets a `False` entry for its `"period_test"` column. 

Now we can use the `"period_test"` variable to remove all products that have not been observed in all three periods. We'll also filter by standard in the `"supp"` column. We will store the remaining products in a new dataframe, `data_c`.

In [ ]:
data_c = data.loc[ data["period_test"] & (data["supp"] == "standard")]
data_c.head()

Now we can calculate the means of `price_per_oz` by grouping the data according to `store_type`, `taxed`, and `time`. We'll use a technique called method chaining to do this: this just means chaining many successive methods together one after another. The first operation groups all of the variables together by `"taxed"`, `"store_type"`, and `"time"`. The second produces some aggregates via `.agg`, namely new columns `avg_price` and `n` that are given via the `("input column", "function to be applied")` syntax. The new dataframe has the groupby variables as its index, so we reset the index (to row numbers) ready to perform a `pivot` to bring the data in the shape that we want.

In [ ]:
table_res = (data_c
    .groupby(["taxed", "store_type", "time"])
    .agg(avg_price = ("price_per_oz", "mean"), n=("price_per_oz", "count"))
    .reset_index()
    .pivot(index=["taxed", "store_type", "n"], columns="time", values="avg_price")
)
table_res

Method chaining is a useful technique for data analysis, but it has pros and cons. If you know what a code is doing, and that it works already, it can be substantially easier to read than successive lines of logic using assignment via `=`. However, if something is going wrong within a long method chain, it can be hard to know where and why, and they don't always encourage modular code written via functions.

You can find a deeper dive into method chaining in the [relevant section](https://aeturrell.github.io/coding-for-economists/data-intro.html#advanced-method-chaining) of [Coding for Economists](https://aeturrell.github.io/coding-for-economists).

Use the [S3 Table](https://tinyco.re/9687764) in the journal paper to check how closely your summary data match those in the paper. You should find that your results for Large Supermarkets and Pharmacies match, but the other store types have discrepancies. In Python walk-through 3.5 we will discuss these differences in more detail.

## Python Walkthrough 3.5

**Making a column chart to compare two groups**

### Calculate price differences by store type

Let’s calculate the two price differences (June 2015 minus December 2014 and March 2016 minus December 2014), and store them as `"d1"` and `"d2"` respectively:

In [ ]:
table_res["d1"] = table_res["JUN2015"] - table_res["DEC2014"]
table_res["d2"] = table_res["MAR2016"] - table_res["DEC2014"]
table_res

### Plot a column chart for average price changes

To display `"d1"` and `"d2"` in a column chart, we will use the **seaborn** and **matplotlib** packages, which we loaded at the start, in Python Walkthrough 3.1. This Let’s start with displaying the average price change from December 2014 to June 2015 (which is stored in `"d1"`).

We will use 

In [ ]:
(ggplot(table_res.reset_index(), 
  aes(fill = "taxed", y = "d1", x = "store_type")) +
  geom_bar(position = "dodge", stat = "identity") +
  # Add the axes labels
  labs(y = "Price change (US$/oz)", x = "Store type") +
  # Add the title and legend labels
  scale_fill_discrete(name = "Beverages") +
  ggtitle("Average price change from Dec 2014 to Jun 2015"))

**Figure 3.2** Average price change from December 2014 to June 2015.

Now we do the same for the price change from Dec 2014 to Mar 2016:

In [ ]:
(ggplot(table_res.reset_index(), 
  aes(fill = "taxed", y = "d2", x = "store_type")) +
  geom_bar(position = "dodge", stat = "identity") +
  # Add the axes labels
  labs(y = "Price change (US$/oz)", x = "Store type") +
  # Add the title and legend labels
  scale_fill_discrete(name = "Beverages") +
  ggtitle("Average price change from Dec 2014 to Mar 2016"))

**Figure 3.3** Average price change from December 2014 to March 2016.

## Extension Python Walkthrough 3.6

**Calculating the p-value for price changes**

In this walk-through, we show the calculations required to obtain the p- values in Table S3 of the Silver et al. (2017) paper. Since the p-values are already provided, this walk-through is only for those who want to see how these p-values were calculated. For the categories of Large Supermarkets and Pharmacies, we conduct a hypothesis test, which tests the null hypothesis that the price difference between June 2015 and December 2014 (and March 2016 and December 2014) for the taxed and untaxed beverages in the two store types could be due to chance.

We are interested in whether the difference in average price between JUN2015 and DEC2014 (or MAR2016 and DEC2014) for one group (say, Large Supermarket and taxed) is zero (i.e. there is no difference in the means of the two populations). Note that we are dealing with paired observations (the same product in both time periods).

Let’s use the price difference between June 2015 and December 2014 in Large Supermarkets for taxed beverages as an example. First, we extract the prices for both periods (the vectors `p_1` and `p_2`) and then calculate the difference, element by element (stored as `"Δ_t"`). Because *all operations assume the same index*, we must also reset the indexes of these sets of values that have been drawn from different rows (dropping the original indexes as we go).

In [ ]:
p_1 = data_c.query("store_type == 'Large supermarket' & taxed == 'taxed' & time == 'DEC2014'")["price_per_oz"].reset_index(drop=True)
p_2 = data_c.query("store_type == 'Large supermarket' & taxed == 'taxed' & time == 'JUN2015'")["price_per_oz"].reset_index(drop=True)

# Price difference for taxed products in large supermarkets
Δ_t = p_2 - p_1
Δ_t.head()

All three new variables are vectors with 36 elements. For `Δ_t` to correctly represent the price difference for a particular product in a particular store, we need to be certain that each element in both vectors corresponds to the same product in the same store. 

An alternative to the above, where one just hopes for the best on an ordered dataset, is to extract

To check that the elements match, we will extract the store and product IDs along with the prices, and compare the ordering in p1_alt and p2_alt.

In [ ]:
alt = (data_c.loc[(data_c["store_type"] == "Large supermarket") & (data_c["taxed"] == "taxed"), :]
             .pivot(["store_id", "product_id"], columns="time", values="price_per_oz"))
alt.head()

Now we can simply subtract columns, preserving index:

In [ ]:
p_1_alt, p_2_alt = alt["DEC2014"], alt["JUN2015"]
Δ_t_alt = p_2_alt - p_1_alt
Δ_t_alt.head()

Note that in this case, we can also see which combination produced this particular value of the `price_per_oz`. This approach guarantees identical ordering because the subtraction would fail if the indices were different.

The average value of the price difference is 0.1312, and our task is to evaluate whether this is likely to be due to sampling variation (given the assumption that there is no difference between the populations) or not. To do this, we can use the `ttest` function, which provides the associated p-value.

In [ ]:
# Recognise that the differences come from paired samples.
pg.ttest(p_2, p_1, paired=True)

Alternatively, we can calculate the respective test statistic manually, which also requires us to calculate the standard error of this value:

In [ ]:
Δ_t_alt.mean() / np.sqrt((Δ_t_alt.var()/len(Δ_t_alt)))

*Another alternative* is to use the ttest function directly on `Δ_t`.

To compare the results to the journal paper, look at the extract from Table S3 (the section on Large Supermarkets) shown in Figure 3.4 below. The cell with ‘**’ shows the mean price difference of 1.31 cents ($0.0131).

| Large supermarkets<br>(n = 6)        	| Taxed beverage price<br>(36 sets) 	| Untaxed beverage price<br>(36 sets) 	| Taxed – untaxed difference 	|
|--------------------------------------	|:---------------------------------:	|:-----------------------------------:	|:--------------------------:	|
|                                      	| cents/oz                          	|                            cents/oz 	|                   cents/oz 	|
| Round 1: December 2014               	|                             15.62 	|                               11.19 	|                            	|
| Round 2: June 2015                   	|                             16.93 	|                               11.48 	|                            	|
| Round 3: March 2016                  	|                             16.68 	|                               11.70 	|                            	|
| Mean change<br>(March 2016–Dec 2014) 	|                    1.07, (p=0.01) 	|                      0.51, (p=0.01) 	|             0.56, (p=0.22) 	|
| Mean change<br>(June 2015–Dec 2014)  	|                 1.31, (p<0.001)** 	|                      0.29, (p=0.08) 	|            1.02, (p=0.002) 	|

**Figure 3.4** Table S3 in Silver et al. (2017), showing means and confidence intervals. n = number of stores of each time

In our test output we get a very small p-value (0.000032) which in the table is indicated by the double asterisk.

Tests for other store types are calculated similarly, by changing the data extracted to `p_1` and `p_2`. Let’s do that for one more example: the price difference between June 2015 and December 2014 in Large Supermarkets for untaxed beverages.

In [ ]:
alt_nt = (data_c.loc[(data_c["store_type"] == "Large supermarket") & (data_c["taxed"] == "not taxed"), :]
             .pivot(["store_id", "product_id"], columns="time", values="price_per_oz"))
Δ_nt_alt = alt_nt["JUN2015"] - alt_nt["DEC2014"]
print(f"Mean difference is {100*Δ_nt_alt.mean():.2f} cents/oz")
pg.ttest(Δ_nt_alt, y=0, paired=True)

You should be able to recognize the mean difference and the p-value in the excerpt of Table S3 provided in Figure 3.4.

Let’s also replicate the last section of Table S3, which shows the difference between the price changes in taxed and untaxed products, that is, we want to know whether `Δ_t_alt` *and* `Δ_nt_alt` have different means. We will apply the two sample hypothesis tests, but this time for unpaired data, as the products differ across samples.

In [ ]:
pg.ttest(Δ_t_alt, Δ_nt_alt)

Again you should be able to identify the corresponding entries in Table S3 shown in Figure 3.4. The main entry in the table is 1.02, indicating that the means of the two groups differ by 1.02 cents. This is confirmed in our calculations, as $0.01312 − $0.00288 is about $0.0102 or 1.02 cents. The p-value of 0.002 is also the same as the one in Table S3.

## Python Walkthrough 3.7

**Importing data from a Stata file and plotting a line chart**

## Import data and create a table of average monthly prices

To import data from a Stata file (`.dta` format) we need the **pandas** package's `pd.read_stata` function. Remember to download the data from [this link](https://www.globalfoodresearchprogram.org/wp-content/uploads/2021/10/Berkeley_data.zip) and save the file with extension `.dta` in the `data/` folder of your working directory.

In [ ]:
path_to_stata_data = Path("data/public_use_weighted_prices.dta")
posd = pd.read_stata(path_to_stata_data)
posd.head()

You will see that for each month and location (Berkeley or Non-Berkeley), there are prices for a variety of beverage categories, and we know whether the category is taxed or not. For any particular time-location-tax status combination we want the average price of all products.

To make the summary table, we use method chaining again:

In [ ]:
table_test = (posd
                .groupby(["year", "month", "location", "tax"])["price"]
                .agg("mean")
                .reset_index())
table_test

Here we used a quick trick to pivot the data so that location appears in two separate columns according to its two separate values. The trick is called `unstack` and it takes values from the index and pastes them across columns. `unstack(2)` takes the position 2 level of the index and turns it into columns (in this case, that's location).

We'll now prepare the data for plotting in a line chart.

First, we reset the index so that it is just composed of row numbers. We'd also like to add a column that tracks time as a time series.

In [ ]:
table_test["date"] = table_test.apply(lambda row: pd.to_datetime(str(int(row["year"])) + "-" + str(int(row["month"])), format="%Y-%m"), axis=1) + pd.offsets.MonthEnd()
table_test.head()

Here, we use an integer version of the year and month together in a string and pass these to `pd.to_datetime` along with the recipe for the format, `%Y-%m` means YYYY-MM format. This produces a date at the start of each month, `pd.offsets.MonthEnd()` finds the last day in the month.

### Plot a line chart

Now we can create a line chart.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3))
# zorder sets which layer appears on top
sns.lineplot(data=table_test, x="date", y="price", hue="tax", style="location", ax=ax, zorder=2)
# Add vertical lines
ax.axvline("2015-01-01", color="k", alpha=0.3, zorder=1, linestyle="--")
ax.axvline("2015-03-01", color="k", alpha=0.3, zorder=1, linestyle="--")
ax.legend(ncol=2, frameon=False)
ax.set_ylabel("Average price")
# \n puts text on a new line
ax.set_title("Average price of taxed and non-taxed beverages \nin Berkeley and non-Berkeley areas")
ax.annotate("Pre-tax", xy=("2014-09-01", 4.5))
ax.annotate("Post-tax", xy=("2015-06-01", 4.5))
plt.show()

**Figure 3.6** Average price of taxed and non-taxed beverages in Berkeley and non-Berkeley areas.